# **0-** Background

- **The Notebook does product recommendation analysis. Using Co-occurence matrices, this code creates a co-occurence metric and a transaction metric**
     - Co-occurrence Count : Refers to how many times Product Focus and the Recommended Product have been bought together by the same customer across different transactions (added across all customers).
     - Transaction Count : Refers to how many times Product Focus and the Recommended Product have been bought together in the same transaction (added across all customers).

## **0.1-** Extra

# **1-** Enviroment Setup

In [1]:
# Install when needed
!pip install openpyxl
#!pip install mlxtend

SyntaxError: invalid syntax (300808291.py, line 2)

In [34]:
import pandas as pd
import numpy as np
from itertools import combinations, product
from collections import defaultdict

# **2-** Data Input (GCS) & Cleaning

In [114]:
# Read the Excel file into a dataframe from GCS
data = pd.read_excel("GCC-KER-Raw-Data-full-cleaned-2023.xlsx", engine='openpyxl') 

In [111]:
# ONLY IF CLEANED
# Import the cleaned data back if kernel is reset and you dont want to restart the cleaning
data = pd.read_excel("YSL - Products Last 24 Months - cleaned.xlsx", engine='openpyxl') 

In [115]:
data[data.columns]

,transid,custid,date,productcode,productname,qty,amount,Year
0,10161,98397,2023-03-01 14:57:36,3474636382408,KER NUTR BAIN MAGISTRAL 250ML VF46,1,138.0,2023
1,10161,98397,2023-03-01 14:57:36,3474636382446,NUTRITIVE FONDANT MAGISTRAL 200ml,1,171.0,2023
2,10161,98397,2023-03-01 14:57:36,3474636857883,K GEN FONDANT 200ML VJ01,1,170.0,2023
3,10161,98397,2023-03-01 14:57:36,3474636728268,K CHRONO BAIN 250ML V352,1,173.0,2023
4,10162,98397,2023-03-01 17:16:42,3474636400195,DISC BAIN FLUIDEALISTE GENTLE 250ml,1,138.0,2023
...,...,...,...,...,...,...,...,...
52963,53364,2242,2023-12-09 09:10:13,3474636728268,KER CHRONO BAIN 250ML V352,2,193.0,2023
52964,53364,2242,2023-12-09 09:10:13,3474636614103,KER ELIX ULT BAIN 250ML VD50,1,169.0,2023
52965,53723,2242,2023-12-19 20:15:11,3474636397969,KER RES BAIN THERAPISTE 250ML VF46,1,161.0,2023
52966,53723,2242,2023-12-19 20:15:11,3474636397983,KER RES MSQ THERAPISTE 200ML VF62,1,266.0,2023


In [ ]:
# Show all unique values of a column:
sorted(data['Price (USD)'].unique())

## **2.1-** Removing rows with a specific value in a specific column

In [72]:
# Identify rows where the specified column matches the given value
matching_rows = data["Item Quantity"] == -1
count_matching = matching_rows.sum()

print(f"Rows that match: {count_matching}")

Rows that match: 270


In [73]:
# Show the dataframe if we removed the matching rows
data[~matching_rows]

,Speedbus ID,Invoice Number,Transaction Date,Item ID,Item Description: Item Description,Item Quantity,Price (USD),Transaction Year
0,9995677404,001216133,2023-01-01,LE111000,LE111000:YSL LB 90ML +10ML+ MINI RPC21 HOL22,1,641.70,2023
1,99916845700,001216245,2023-01-01,LE113000,LE113000:Y EDP100ML+EDP10ML+ASB50ML HOL22,1,621.00,2023
2,9993564803,001216272,2023-01-01,LA647900,LA647900:COUTURE EYELINER VINYL 01,1,122.50,2023
3,9993564803,001216272,2023-01-01,L7847500,L7847500:MON PARIS EDP 50ML,1,445.00,2023
4,99916700538,001216369,2023-01-01,L7976200,L7976200:LVPC UNIVERSITE EDP V125ML,1,1138.50,2023
...,...,...,...,...,...,...,...,...
46333,99918138109,75708301202312261431,2023-12-26,034816100250,LD857100:YSL LIBRE LE PARFUM S50ML,1,161.03,2023
46334,99918128984,75708301202312271440,2023-12-27,034816525394,LE227400:YSL LIBRE ABSOLU PLATINE 90ML,1,251.42,2023
46335,99915317076,75708301202312271441,2023-12-27,034816100247,LD857200:YSL LIBRE LE PARFUM S90ML,1,208.49,2023
46336,99915439997,75708301202312281452,2023-12-28,034816598044,LE652100:YSL LB EDP 90ML+BL50ML+10ML HOL23,1,161.30,2023


In [74]:
# Remove the rows and return the modified DataFrame
data = data[~matching_rows]

In [75]:
data.reset_index(drop=True, inplace=True)

In [76]:
data[data.columns]

,Speedbus ID,Invoice Number,Transaction Date,Item ID,Item Description: Item Description,Item Quantity,Price (USD),Transaction Year
0,9995677404,001216133,2023-01-01,LE111000,LE111000:YSL LB 90ML +10ML+ MINI RPC21 HOL22,1,641.70,2023
1,99916845700,001216245,2023-01-01,LE113000,LE113000:Y EDP100ML+EDP10ML+ASB50ML HOL22,1,621.00,2023
2,9993564803,001216272,2023-01-01,LA647900,LA647900:COUTURE EYELINER VINYL 01,1,122.50,2023
3,9993564803,001216272,2023-01-01,L7847500,L7847500:MON PARIS EDP 50ML,1,445.00,2023
4,99916700538,001216369,2023-01-01,L7976200,L7976200:LVPC UNIVERSITE EDP V125ML,1,1138.50,2023
...,...,...,...,...,...,...,...,...
46063,99918138109,75708301202312261431,2023-12-26,034816100250,LD857100:YSL LIBRE LE PARFUM S50ML,1,161.03,2023
46064,99918128984,75708301202312271440,2023-12-27,034816525394,LE227400:YSL LIBRE ABSOLU PLATINE 90ML,1,251.42,2023
46065,99915317076,75708301202312271441,2023-12-27,034816100247,LD857200:YSL LIBRE LE PARFUM S90ML,1,208.49,2023
46066,99915439997,75708301202312281452,2023-12-28,034816598044,LE652100:YSL LB EDP 90ML+BL50ML+10ML HOL23,1,161.30,2023


In [ ]:
# If needed
data.drop('Item Quantity', axis=1, inplace=True)

In [77]:
# Save the DataFrame to an Excel file if needed
data.to_excel('YSL - Products 2023 - cleaned.xlsx', engine='openpyxl', index=False)

# **3-** Analysis

In [116]:
print(data['productcode'].value_counts())

3474636613908    2777
3474636858033    1466
3474636857883    1301
3474636858002    1281
3474636397983    1249
                 ... 
3474636382668       1
3474637052201       1
3474636349913       1
3474636397372       1
3474636494934       1
Name: productcode, Length: 228, dtype: int64


In [117]:
# THIS CODE CALCS CO-OCCURENCE BASED ON CUSTOMER HISTORICAL PURCHASES = HOW MANY 
# TIMES TWO PRODUCTS APPEARED IN THE SAME HISTORY OF A CUSTOMER COUNTED ACCROSS ALL CUSTOMERS

# AND

# ANOTHER METRIC IS TRANSACTION COUNT = WHERE IT COUNTS HOW MANY TIMES THE PAIR APPEARED IN THE SAME TRANSACTION FOR THE SAME CUSTOMER


# Group by Invoice Number and Speedbus ID to find items bought in the same transaction
transaction_items = data.groupby(['transid', 'custid'])['productcode'].apply(list)

# Calculate co-occurrence within the same transaction
transaction_co_occurrence = defaultdict(int)
for items in transaction_items:
    for item_pair in combinations(set(items), 2):
        transaction_co_occurrence[item_pair] += 1
        transaction_co_occurrence[item_pair[::-1]] += 1

# Convert transaction co-occurrence counts to a DataFrame
transaction_co_occurrence_df = pd.DataFrame(((pair[0], pair[1], count) for pair, count in transaction_co_occurrence.items()), columns=['Item A', 'Item B', 'Transaction Count'])

# integrate the original co-occurrence count
# Calculate co-occurrence across all customer purchases, regardless of transaction
customer_purchases = data.groupby('custid')['productcode'].apply(list)
overall_co_occurrence = defaultdict(int)
for items in customer_purchases:
    for item_pair in combinations(set(items), 2):
        overall_co_occurrence[item_pair] += 1
        overall_co_occurrence[item_pair[::-1]] += 1

# Convert overall co-occurrence counts to a DataFrame
overall_co_occurrence_df = pd.DataFrame(((pair[0], pair[1], count) for pair, count in overall_co_occurrence.items()), columns=['Item A', 'Item B', 'Co-occurrence Count'])

# Merge the two datasets on Item A and Item B
merged_df = pd.merge(transaction_co_occurrence_df, overall_co_occurrence_df, on=['Item A', 'Item B'])

# Normalize scores (optional step here based on the new counts)
# You can normalize either or both count columns as needed

# Sort and select top 5 recommendations based on Co-occurrence Count
recommendations = merged_df.sort_values(by=['Item A', 'Co-occurrence Count', 'Transaction Count'], ascending=[True, False, False]).groupby('Item A').head(5)

In [118]:
# Export the recommendations to a CSV file
# recommendations.to_csv('product_bundling_recommendations_with_co_occurrences.csv', index=False)
recommendations.to_excel('GCC-KER-Raw-Data-full-cleaned-2023-bundling.xlsx', engine='openpyxl', index=False)

# **A-** Other

In [123]:
# THIS CODE CALCS CO-OCCURENCE BASED ON CUSTOMER HISTORICAL PURCHASES = HOW MANY 
# TIMES TWO PRODUCTS APPEARED IN THE SAME HISTORY OF A CUSTOMER COUNTED ACCROSS ALL CUSTOMERS


# Create a mapping of Speedbus ID to items purchased
customer_purchases = data.groupby('Speedbus ID')['Item ID'].apply(list)

# Calculate co-occurrence of each item pair within the same transaction
co_occurrence = defaultdict(int)
for items in customer_purchases:
    for item_pair in combinations(set(items), 2):
        co_occurrence[item_pair] += 1
        co_occurrence[item_pair[::-1]] += 1

# Convert the co-occurrence counts to a DataFrame
co_occurrence_df = pd.DataFrame(((pair[0], pair[1], count) for pair, count in co_occurrence.items()), columns=['Item A', 'Item B', 'Count'])

# You may want to normalize the scores here based on your specific criteria
# For now, let's assume we're directly using counts as a proxy for scoring

# To normalize scores between 0 and 1, find the max count
max_count = co_occurrence_df['Count'].max()
co_occurrence_df['Score'] = co_occurrence_df['Count'] / max_count

# Now, for each item, select the top 5 items to bundle with based on the score
recommendations = co_occurrence_df.sort_values(by=['Item A', 'Score'], ascending=[True, False]).groupby('Item A').head(5)

# Export the recommendations to a CSV file
recommendations.to_csv('product_bundling_recommendations.csv', index=False)


In [129]:
# THIS CODE CALCS CO-OCCURENCE BASED ON CUSTOMER HISTORICAL PURCHASES = HOW MANY 
# TIMES TWO PRODUCTS APPEARED IN THE SAME HISTORY OF A CUSTOMER COUNTED ACCROSS ALL CUSTOMERS

# AND

# ANOTHER METRIC IS TRANSACTION COUNT = WHERE IT COUNTS HOW MANY TIMES THE PAIR APPEARED IN THE SAME TRANSACTION FOR THE SAME CUSTOMER


# Group by Invoice Number and Speedbus ID to find items bought in the same transaction
transaction_items = data.groupby(['Invoice Number', 'Speedbus ID'])['Item ID'].apply(list)

# Calculate co-occurrence within the same transaction
transaction_co_occurrence = defaultdict(int)
for items in transaction_items:
    for item_pair in combinations(set(items), 2):
        transaction_co_occurrence[item_pair] += 1
        transaction_co_occurrence[item_pair[::-1]] += 1

# Convert transaction co-occurrence counts to a DataFrame
transaction_co_occurrence_df = pd.DataFrame(((pair[0], pair[1], count) for pair, count in transaction_co_occurrence.items()), columns=['Item A', 'Item B', 'Transaction Count'])

# Now, integrate the original co-occurrence count
# Calculate co-occurrence across all customer purchases, regardless of transaction
customer_purchases = data.groupby('Speedbus ID')['Item ID'].apply(list)
overall_co_occurrence = defaultdict(int)
for items in customer_purchases:
    for item_pair in combinations(set(items), 2):
        overall_co_occurrence[item_pair] += 1
        overall_co_occurrence[item_pair[::-1]] += 1

# Convert overall co-occurrence counts to a DataFrame
overall_co_occurrence_df = pd.DataFrame(((pair[0], pair[1], count) for pair, count in overall_co_occurrence.items()), columns=['Item A', 'Item B', 'Co-occurrence Count'])

# Merge the two datasets on Item A and Item B
merged_df = pd.merge(transaction_co_occurrence_df, overall_co_occurrence_df, on=['Item A', 'Item B'])

# Normalize scores (optional step here based on the new counts)
# You can normalize either or both count columns as needed

# Sort and select top 5 recommendations based on Co-occurrence Count
recommendations = merged_df.sort_values(by=['Item A', 'Co-occurrence Count', 'Transaction Count'], ascending=[True, False, False]).groupby('Item A').head(5)

# Export the recommendations to a CSV file
recommendations.to_csv('product_bundling_recommendations_with_co_occurrences.csv', index=False)

## **B-** Old

In [61]:
# Column names can be changed as needed
#user_id_col = 'Speedbus ID'  # Change this to the name of your user ID column
#item_id_col = 'Item ID'  # Change this to the name of your item ID column

In [ ]:
# Library Importing
import pandas as pd
from collections import defaultdict

# Load the dataset from GCS into a Data frame
df = pd.read_csv("My_Product_Performance_v2022.csv")

# Co-Occurance matrix creation
def create_co_occurrence_matrix(df):
    user_items = defaultdict(set)
    co_occurrence = defaultdict(int)

    for index, row in df.iterrows():
        user_items[row["user_id"]].add(row["item_id"])

    for item_set in user_items.values():
        for item1 in item_set:
            for item2 in item_set:
                if item1 != item2:
                    co_occurrence[(item1, item2)] += 1

    return co_occurrence

def find_similar_items(item_id, co_occurrence_matrix):
    similar_items = defaultdict(int)

    for (item1, item2), count in co_occurrence_matrix.items():
        if item1 == item_id or item2 == item_id:
            similar_items[item2 if item1 == item_id else item1] = count

    return dict(sorted(similar_items.items(), key=lambda x: x[1], reverse=True))


def generate_recommendations(user_id, co_occurrence_matrix, df, top_n=5):
    user_items = df[df["user_id"] == user_id]["item_id"].tolist()
    recommendations = []

    similar_items = defaultdict(int)
    for item in user_items:
        item_similarities = find_similar_items(item, co_occurrence_matrix)
        for similar_item, count in item_similarities.items():
            if similar_item not in user_items:
                similar_items[similar_item] += count

    # Sort similar items by count
    sorted_similar_items = dict(sorted(similar_items.items(), key=lambda x: x[1], reverse=True))

    # Generate recommendations
    for similar_item, _ in sorted_similar_items.items():
        recommendations.append(similar_item)
        if len(recommendations) >= top_n:
            break

    return recommendations


def display_co_occurrence_sample(co_occurrence_matrix, sample_size=10):
    print("Sample of item-item co-occurrence matrix:")
    for i, ((item1, item2), count) in enumerate(co_occurrence_matrix.items()):
        if i >= sample_size:
            break
        print(f"({item1}, {item2}): {count}")


def co_occurrence_matrix_to_df(co_occurrence_matrix):
    co_occurrence_list = [{"item1": item1, "item2": item2, "count": count}
                           for (item1, item2), count in co_occurrence_matrix.items()]
    return pd.DataFrame(co_occurrence_list)


def save_co_occurrence_matrix_to_csv(co_occurrence_matrix, output_filename):
    co_occurrence_df = co_occurrence_matrix_to_df(co_occurrence_matrix)
    co_occurrence_df.to_csv(output_filename, index=False)
    print(f"Co-occurrence matrix has been saved to {output_filename}")

# Apply the algorithm
co_occurrence_matrix = create_co_occurrence_matrix(df)

# Display a sample of the co-occurrence matrix
display_co_occurrence_sample(co_occurrence_matrix)

# Export the full co-occurrence matrix to a CSV file
output_filename = "co_occurrence_matrix.csv"
save_co_occurrence_matrix_to_csv(co_occurrence_matrix, output_filename)

# Generate recommendations for a specific user
# user_id = 1669640  # Set the user_id for which you want to generate recommendations

# recommendations = generate_recommendations(user_id, co_occurrence_matrix, df)

# Print the recommendations
# print(f"\nTop {len(recommendations)} recommendations for user {user_id}:")
# for i, rec_item_id in enumerate(recommendations, start=1):
    # print(f"{i}. Item ID: {rec_item_id}")